<a href="https://colab.research.google.com/github/christophergaughan/ChristopherGaughan.io/blob/master/7_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Define Functions to Query and Download Data from the GDC API

In [ ]:
import requests
import json
import os
import shutil

# Define the directory to save downloaded files
directory_path = '/content/drive/MyDrive/Colab Notebooks/gdc2_files'

# Clear the directory if it exists
if os.path.exists(directory_path):
    shutil.rmtree(directory_path)

# Recreate the directory
os.makedirs(directory_path)

# Function to query the GDC API
def query_gdc_api():
    url = "https://api.gdc.cancer.gov/files"

    # Define the parameters for the query
    params = {
        "filters": json.dumps({
            "op": "and",
            "content": [
                {
                    "op": "in",
                    "content": {
                        "field": "cases.project.project_id",
                        "value": ["TCGA-BRCA"]
                    }
                },
                {
                    "op": "in",
                    "content": {
                        "field": "data_category",
                        "value": ["Transcriptome Profiling"]
                    }
                },
                {
                    "op": "in",
                    "content": {
                        "field": "data_type",
                        "value": ["Gene Expression Quantification"]
                    }
                }
            ]
        }),
        "fields": "file_id,file_name,cases.submitter_id",
        "format": "json",
        "size": "500"  # Increase the size to 500
    }

    response = requests.get(url, params=params)
    print(f"Query URL: {response.url}")  # Print the query URL for debugging
    if response.status_code == 200:
        data = response.json()
        return data['data']['hits']
    else:
        print(f"Error querying GDC API: {response.status_code}")
        return []

# Query the GDC API and print the retrieved files
files = query_gdc_api()
for file in files:
    print(f"Retrieved file: {file['file_name']} with ID: {file['file_id']}")

# Ensure files are retrieved before attempting to download
if not files:
    print("No files retrieved. Please check the query parameters.")
else:
    print(f"Total files retrieved: {len(files)}")

# Function to download files from the GDC API
def download_files(files):
    for file in files:
        file_id = file['file_id']
        file_name = file['file_name']
        file_path = os.path.join(directory_path, file_name)

        if not os.path.exists(file_path):
            download_url = f"https://api.gdc.cancer.gov/data/{file_id}"
            response = requests.get(download_url, stream=True)
            if response.status_code == 200:
                with open(file_path, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=1024):
                        if chunk:
                            f.write(chunk)
                print(f"Downloaded {file_name}")
            else:
                print(f"Failed to download {file_name}")

# Download the files
download_files(files)


**preprocess the RNA-Seq files:**

In [ ]:
import pandas as pd
import json
import os

# Function to preprocess RNA-Seq files
def preprocess_rna_seq_file(file_path):
    try:
        # Read the file with proper column names and skip the initial rows if they don't conform to the structure
        df = pd.read_csv(file_path, sep='\t', comment='#')

        # Display the first few rows to understand the structure
        print(f"First few rows of the file {file_path}:")
        print(df.head())

        # Remove rows with NaN in gene_name or gene_type
        df = df.dropna(subset=['gene_name', 'gene_type'])

        # Filter out rows with gene_id values like 'N_unmapped', 'N_multimapping', etc.
        df = df[~df['gene_id'].str.contains('N_unmapped|N_multimapping|N_noFeature|N_ambiguous')]

        if df.empty or df.shape[1] == 0:
            raise ValueError("File is empty or has no valid columns")

        return df
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return None

# Directory path for downloaded files
directory_path = '/content/drive/MyDrive/Colab Notebooks/gdc2_files'
file_ids = [f for f in os.listdir(directory_path) if f.endswith('.rna_seq.augmented_star_gene_counts.tsv')]

# List to hold all DataFrames
rna_seq_dfs = []

# Preprocess and collect all RNA-Seq DataFrames
for file_id in file_ids:
    file_path = os.path.join(directory_path, file_id)
    rna_seq_df = preprocess_rna_seq_file(file_path)
    if rna_seq_df is not None:
        rna_seq_df['submitter_id'] = file_id.split('.')[0]  # Add submitter_id for merging
        rna_seq_dfs.append(rna_seq_df)

# Concatenate all RNA-Seq DataFrames
if rna_seq_dfs:
    all_rna_seq_df = pd.concat(rna_seq_dfs, ignore_index=True)
    # Display the combined DataFrame
    print(all_rna_seq_df.head())
    print(all_rna_seq_df.info())
else:
    print("No valid RNA-Seq data found.")


In [ ]:
import os
import pandas as pd

# Function to preprocess RNA-Seq files
def preprocess_rna_seq_file(file_path):
    try:
        # Read the file and handle comment lines
        df = pd.read_csv(file_path, sep='\t', comment='#')

        # Remove rows with NaN in 'gene_name' or 'gene_type'
        df = df.dropna(subset=['gene_name', 'gene_type'])

        # Filter out rows with gene_id values like 'N_unmapped', 'N_multimapping', etc.
        df = df[~df['gene_id'].str.contains('N_unmapped|N_multimapping|N_noFeature|N_ambiguous')]

        if df.empty or df.shape[1] == 0:
            raise ValueError("File is empty or has no valid columns")

        return df
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return None

# Directory path for downloaded files
directory_path = '/content/drive/MyDrive/Colab Notebooks/gdc2_files'
file_ids = [f for f in os.listdir(directory_path) if f.endswith('.rna_seq.augmented_star_gene_counts.tsv')]

# List to hold all DataFrames
rna_seq_dfs = []

# Preprocess and collect all RNA-Seq DataFrames
for file_id in file_ids:
    file_path = os.path.join(directory_path, file_id)
    rna_seq_df = preprocess_rna_seq_file(file_path)
    if rna_seq_df is not None:
        rna_seq_df['submitter_id'] = file_id.split('.')[0]  # Add submitter_id for merging
        rna_seq_dfs.append(rna_seq_df)

# Concatenate all RNA-Seq DataFrames
if rna_seq_dfs:
    all_rna_seq_df = pd.concat(rna_seq_dfs, ignore_index=True)
    # Display the combined DataFrame
    print(all_rna_seq_df.head())
    print(all_rna_seq_df.info())
else:
    print("No valid RNA-Seq data found.")


In [ ]:
# Display summary statistics
summary_stats = all_rna_seq_df.describe()
print(summary_stats)


In [ ]:
# Filter genes with TPM > 10
filtered_df = all_rna_seq_df[all_rna_seq_df['tpm_unstranded'] > 10]
print(filtered_df.head())


In [ ]:
# Export the combined DataFrame to a CSV file
output_file_path = '/content/drive/MyDrive/Colab Notebooks/processed_rna_seq_data.csv'
all_rna_seq_df.to_csv(output_file_path, index=False)
print(f"Data exported to {output_file_path}")


In [ ]:
import os
import pandas as pd

# Function to preprocess RNA-Seq files
def preprocess_rna_seq_file(file_path):
    try:
        # Read the file and handle comment lines
        df = pd.read_csv(file_path, sep='\t', comment='#')

        # Remove rows with NaN in 'gene_name' or 'gene_type'
        df = df.dropna(subset=['gene_name', 'gene_type'])

        # Filter out rows with gene_id values like 'N_unmapped', 'N_multimapping', etc.
        df = df[~df['gene_id'].str.contains('N_unmapped|N_multimapping|N_noFeature|N_ambiguous')]

        if df.empty or df.shape[1] == 0:
            raise ValueError("File is empty or has no valid columns")

        return df
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return None

# Directory path for downloaded files
directory_path = '/content/drive/MyDrive/Colab Notebooks/gdc2_files'
file_ids = [f for f in os.listdir(directory_path) if f.endswith('.rna_seq.augmented_star_gene_counts.tsv')]

# List to hold all DataFrames
rna_seq_dfs = []

# Preprocess and collect all RNA-Seq DataFrames
for file_id in file_ids:
    file_path = os.path.join(directory_path, file_id)
    rna_seq_df = preprocess_rna_seq_file(file_path)
    if rna_seq_df is not None:
        rna_seq_df['submitter_id'] = file_id.split('.')[0]  # Add submitter_id for merging
        rna_seq_dfs.append(rna_seq_df)

# Concatenate all RNA-Seq DataFrames
if rna_seq_dfs:
    all_rna_seq_df = pd.concat(rna_seq_dfs, ignore_index=True)
    # Display the combined DataFrame
    print(all_rna_seq_df.head())
    print(all_rna_seq_df.info())

    # Generate summary statistics
    summary_stats = all_rna_seq_df.describe()
    print(summary_stats)

    # Filter genes with TPM > 10
    filtered_df = all_rna_seq_df[all_rna_seq_df['tpm_unstranded'] > 10]
    print(filtered_df.head())

    # Export the combined DataFrame to a CSV file
    output_file_path = '/content/drive/MyDrive/Colab Notebooks/processed_rna_seq_data.csv'
    all_rna_seq_df.to_csv(output_file_path, index=False)
    print(f"Data exported to {output_file_path}")
else:
    print("No valid RNA-Seq data found.")


In [ ]:
import pandas as pd

# Load the data
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/processed_rna_seq_data.csv')

# Check for missing values
print(data.isnull().sum())

# Summary statistics
print(data.describe())


## Simplified Exploratory Data Analysis (EDA)

1. **Distribution Analysis**
Combine the histograms and boxplots into a single function for simplicity.



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Function to plot distributions
def plot_distributions(data, columns, sample_size=100000):
    # Sample the data for quicker processing
    sampled_data = data.sample(n=sample_size, random_state=42)

    for column in columns:
        plt.figure(figsize=(14, 6))

        # Plot histogram
        plt.subplot(1, 2, 1)
        sns.histplot(sampled_data[column], kde=True, bins=50)  # Limiting bins for performance
        plt.title(f'Distribution of {column}')
        plt.xlabel(column)
        plt.ylabel('Frequency')

        # Plot boxplot
        plt.subplot(1, 2, 2)
        sns.boxplot(y=sampled_data[column])
        plt.title(f'Boxplot of {column}')

        plt.tight_layout()
        plt.show()

# Columns to visualize
columns_to_plot = ['tpm_unstranded', 'fpkm_unstranded']

# Plot the distributions with sampling
plot_distributions(data, columns_to_plot)


The distribution of tpm_unstranded is heavily skewed with many values close to zero and some extremely high values. The boxplot also shows the presence of numerous outliers.

To get more insights from the data, we can perform the following steps:

1. **Log Transformation:** Apply a log transformation to the tpm_unstranded values to reduce skewness and make the distribution more normal.
2. **Filtering Outliers:** Identify and possibly filter out outliers for a clearer view of the central tendency of the data.
3. **Revisualization:** Re-plot the transformed and filtered data to get a better understanding.


## Applying Log Transformation and Filtering Outliers

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Function to plot distributions with transformations
def plot_transformed_distributions(data, column, sample_size=100000):
    # Sample the data for quicker processing
    sampled_data = data.sample(n=sample_size, random_state=42)

    # Apply log transformation (adding a small constant to avoid log(0))
    sampled_data['log_' + column] = np.log1p(sampled_data[column])

    plt.figure(figsize=(14, 6))

    # Plot histogram of log-transformed data
    plt.subplot(1, 2, 1)
    sns.histplot(sampled_data['log_' + column], kde=True, bins=50)  # Limiting bins for performance
    plt.title(f'Log-Transformed Distribution of {column}')
    plt.xlabel('log(' + column + ')')
    plt.ylabel('Frequency')

    # Plot boxplot of log-transformed data
    plt.subplot(1, 2, 2)
    sns.boxplot(y=sampled_data['log_' + column])
    plt.title(f'Boxplot of Log-Transformed {column}')

    plt.tight_layout()
    plt.show()

# Column to visualize
column_to_plot = 'tpm_unstranded'

# Plot the log-transformed distributions
plot_transformed_distributions(data, column_to_plot)


The log-transformed distribution and boxplot for tpm_unstranded provide a clearer view of the data:

Log-Transformed Distribution: The distribution is still right-skewed but much less so than the original distribution. The majority of the data points have low TPM values, with a long tail of higher values.
Boxplot: The boxplot shows that even after log transformation, there are still many outliers.


In [ ]:
# Identify the top 10 most highly expressed genes
top_expressed_genes = data[['gene_id', 'gene_name', 'tpm_unstranded']].nlargest(10, 'tpm_unstranded')
print("Top 10 most highly expressed genes:")
print(top_expressed_genes)

# Identify the top 10 least expressed genes (excluding zeros)
least_expressed_genes = data[['gene_id', 'gene_name', 'tpm_unstranded']][data['tpm_unstranded'] > 0].nsmallest(10, 'tpm_unstranded')
print("\nTop 10 least expressed genes (excluding zeros):")
print(least_expressed_genes)


In [ ]:
print(data.columns)



In [ ]:
# Identify the top 10 most highly expressed genes
top_expressed_genes = data[['gene_id', 'gene_name', 'tpm_unstranded']].nlargest(10, 'tpm_unstranded')
print("Top 10 most highly expressed genes:")
print(top_expressed_genes)

# Identify the top 10 least expressed genes (excluding zeros)
least_expressed_genes = data[['gene_id', 'gene_name', 'tpm_unstranded']][data['tpm_unstranded'] > 0].nsmallest(10, 'tpm_unstranded')
print("\nTop 10 least expressed genes (excluding zeros):")
print(least_expressed_genes)


In [ ]:
import numpy as np

# Simulate condition data (e.g., two conditions 'A' and 'B')
np.random.seed(42)  # For reproducibility
data['condition'] = np.random.choice(['A', 'B'], size=len(data))

# Calculate mean TPM for each gene per condition
mean_tpm_per_condition = data.groupby(['gene_id', 'gene_name', 'condition'])['tpm_unstranded'].mean().unstack()

# Display the mean TPM per condition for the first few genes
print(mean_tpm_per_condition.head())


In [ ]:
# Identify the top 10 most highly expressed genes for each condition
top_expressed_genes_A = mean_tpm_per_condition['A'].nlargest(10)
top_expressed_genes_B = mean_tpm_per_condition['B'].nlargest(10)

print("Top 10 most highly expressed genes in condition A:")
print(top_expressed_genes_A)

print("\nTop 10 most highly expressed genes in condition B:")
print(top_expressed_genes_B)

# Identify the top 10 least expressed genes (excluding zeros) for each condition
least_expressed_genes_A = mean_tpm_per_condition['A'][mean_tpm_per_condition['A'] > 0].nsmallest(10)
least_expressed_genes_B = mean_tpm_per_condition['B'][mean_tpm_per_condition['B'] > 0].nsmallest(10)

print("\nTop 10 least expressed genes in condition A (excluding zeros):")
print(least_expressed_genes_A)

print("\nTop 10 least expressed genes in condition B (excluding zeros):")
print(least_expressed_genes_B)


In [ ]:
# Calculate fold change
mean_tpm_per_condition['fold_change'] = mean_tpm_per_condition['B'] / mean_tpm_per_condition['A']

# Log2 Fold Change (optional for better interpretation)
mean_tpm_per_condition['log2_fold_change'] = np.log2(mean_tpm_per_condition['fold_change'])

# Display the data with fold change
print(mean_tpm_per_condition[['A', 'B', 'fold_change', 'log2_fold_change']].head())


In [ ]:
# Replace zero values with a small number
mean_tpm_per_condition.replace(0, np.nan, inplace=True)
mean_tpm_per_condition['fold_change'] = mean_tpm_per_condition['B'] / mean_tpm_per_condition['A']
mean_tpm_per_condition['log2_fold_change'] = np.log2(mean_tpm_per_condition['fold_change'])
mean_tpm_per_condition.replace(np.nan, 0, inplace=True)


In [ ]:
# Define fold-change threshold
fc_threshold = 2

# Identify upregulated genes (log2 fold-change > 1)
upregulated_genes = mean_tpm_per_condition[mean_tpm_per_condition['log2_fold_change'] > 1]

# Identify downregulated genes (log2 fold-change < -1)
downregulated_genes = mean_tpm_per_condition[mean_tpm_per_condition['log2_fold_change'] < -1]

print("Upregulated genes (log2 fold-change > 1):")
print(upregulated_genes)

print("\nDownregulated genes (log2 fold-change < -1):")
print(downregulated_genes)


In [ ]:
# Define the paths
upregulated_path = '/content/drive/MyDrive/Colab Notebooks/upregulated_genes.csv'
downregulated_path = '/content/drive/MyDrive/Colab Notebooks/downregulated_genes.csv'

# Save the results to CSV files
upregulated_genes.to_csv(upregulated_path, header=True)
downregulated_genes.to_csv(downregulated_path, header=True)


In [ ]:
import numpy as np

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Replace zero values with a small number to avoid log transformation issues
mean_tpm_per_condition.replace(0, np.nan, inplace=True)

# Calculate fold change and log2 fold change
mean_tpm_per_condition['fold_change'] = mean_tpm_per_condition['B'] / mean_tpm_per_condition['A']
mean_tpm_per_condition['log2_fold_change'] = np.log2(mean_tpm_per_condition['fold_change'])

# Replace NaN values back with zero for meaningful interpretation
mean_tpm_per_condition.replace(np.nan, 0, inplace=True)

# Display the data with fold change
print(mean_tpm_per_condition[['A', 'B', 'fold_change', 'log2_fold_change']].head())

# Define fold-change threshold
fc_threshold = 2

# Identify upregulated genes (log2 fold-change > 1)
upregulated_genes = mean_tpm_per_condition[mean_tpm_per_condition['log2_fold_change'] > 1]

# Identify downregulated genes (log2 fold-change < -1)
downregulated_genes = mean_tpm_per_condition[mean_tpm_per_condition['log2_fold_change'] < -1]

print("Upregulated genes (log2 fold-change > 1):")
print(upregulated_genes)

print("\nDownregulated genes (log2 fold-change < -1):")
print(downregulated_genes)

# Define the paths
upregulated_path = '/content/drive/MyDrive/Colab Notebooks/upregulated_genes.csv'
downregulated_path = '/content/drive/MyDrive/Colab Notebooks/downregulated_genes.csv'

# Save the results to CSV files
upregulated_genes.to_csv(upregulated_path, header=True)
downregulated_genes.to_csv(downregulated_path, header=True)


In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming `mean_tpm_per_condition` is the DataFrame containing your data
# with 'A' and 'B' as conditions and 'gene_id', 'gene_name' as indices

# Prepare the data
pca_data = mean_tpm_per_condition[['A', 'B']].fillna(0)

# Standardize the data
scaler = StandardScaler()
pca_data_scaled = scaler.fit_transform(pca_data)

# Apply PCA
pca = PCA(n_components=2)
pca_result = pca.fit_transform(pca_data_scaled)

# Create a DataFrame with the PCA results
pca_df = pd.DataFrame(data=pca_result, columns=['PC1', 'PC2'])
pca_df['gene_id'] = mean_tpm_per_condition.index.get_level_values('gene_id')
pca_df['gene_name'] = mean_tpm_per_condition.index.get_level_values('gene_name')

# Plot the PCA results
plt.figure(figsize=(10, 7))
sns.scatterplot(x='PC1', y='PC2', data=pca_df)
plt.title('PCA of RNA-seq Data')
plt.xlabel(f'Principal Component 1 ({pca.explained_variance_ratio_[0]*100:.2f}%)')
plt.ylabel(f'Principal Component 2 ({pca.explained_variance_ratio_[1]*100:.2f}%)')
plt.show()


##  Identify Outliers

In [ ]:
import numpy as np

# Add labels to the PCA DataFrame
pca_df['PC1'] = pca_result[:, 0]
pca_df['PC2'] = pca_result[:, 1]

# Define a threshold to identify outliers (e.g., top 1% of PC1 values)
threshold = np.percentile(pca_df['PC1'], 99)

# Filter the outliers
outliers = pca_df[pca_df['PC1'] > threshold]

# Plot the PCA results with outliers highlighted
plt.figure(figsize=(10, 7))
sns.scatterplot(x='PC1', y='PC2', data=pca_df, label='Data Points')
sns.scatterplot(x='PC1', y='PC2', data=outliers, color='red', label='Outliers')
plt.title('PCA of RNA-seq Data with Outliers Highlighted')
plt.xlabel(f'Principal Component 1 ({pca.explained_variance_ratio_[0]*100:.2f}%)')
plt.ylabel(f'Principal Component 2 ({pca.explained_variance_ratio_[1]*100:.2f}%)')
plt.legend()
plt.show()


In [ ]:
# Extract outliers based on the threshold defined earlier
threshold = np.percentile(pca_df['PC1'], 99)
outliers = pca_df[pca_df['PC1'] > threshold]

# Save outliers to a CSV file for further investigation
outliers.to_csv('/content/drive/MyDrive/Colab Notebooks/rna_seq_outliers.csv', index=False)


In [ ]:
import statsmodels.api as sm

# Assuming you have a DataFrame with expression data for conditions A and B
expression_data = mean_tpm_per_condition.loc[outliers.index]

# Perform differential expression analysis (e.g., using a linear model)
# This is a placeholder for a more complex analysis pipeline
results = []
for gene in expression_data.index:
    model = sm.OLS(expression_data.loc[gene, 'A'], expression_data.loc[gene, 'B'])
    results.append(model.fit())

# Summarize the results
summary = [result.summary() for result in results]


In [ ]:
!pip install statsmodels


In [ ]:
import statsmodels.api as sm
import numpy as np

# Ensure PCA DataFrame has a named index for gene_id
pca_df.index.name = 'gene_id'

# Extract outlier gene IDs based on the threshold defined earlier
threshold = np.percentile(pca_df['PC1'], 99)
outliers = pca_df[pca_df['PC1'] > threshold]

# Extract the gene IDs of the outliers
outlier_gene_ids = outliers.index.unique()

# Filter the expression data for the outlier gene IDs, ensuring alignment
common_gene_ids = mean_tpm_per_condition.index.intersection(outlier_gene_ids)
expression_data = mean_tpm_per_condition.loc[common_gene_ids]

# Perform differential expression analysis using statsmodels
results = []
for gene in expression_data.index:
    try:
        # Use log-transformed data to stabilize variance
        y = np.log2(expression_data.loc[gene, 'B'] + 1)  # Dependent variable
        x = np.log2(expression_data.loc[gene, 'A'] + 1)  # Independent variable
        x = sm.add_constant(x)  # Add intercept
        model = sm.OLS(y, x)
        results.append(model.fit())
    except Exception as e:
        print(f"Could not fit model for gene {gene}: {e}")

# Summarize the results
summary = [result.summary() for result in results if result]

# Print a summary for the first gene as an example
if summary:
    print(summary[0])


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind_from_stats, t

# Sample data for demonstration
np.random.seed(42)
mean_tpm_per_condition = pd.DataFrame({
    'A': np.random.rand(100) * 100,
    'B': np.random.rand(100) * 100
}, index=[f'gene_{i}' for i in range(100)])

# Introduce artificial outliers for demonstration
mean_tpm_per_condition.loc[::10, 'A'] = mean_tpm_per_condition.loc[::10, 'A'] * 10
mean_tpm_per_condition.loc[::10, 'B'] = mean_tpm_per_condition.loc[::10, 'B'] * 10

# Print the first few rows to confirm the data structure
print("First few rows of mean_tpm_per_condition:")
print(mean_tpm_per_condition.head())

# Calculate log2 fold change and p-values for volcano plot
volcano_data = {
    'gene_id': [],
    'log2_fold_change': [],
    'p_value': []
}

# Assume a standard error for both conditions
standard_error = 1.0  # This value is arbitrary for the demonstration

# Iterate over genes and calculate log2 fold change and p-values
for gene in mean_tpm_per_condition.index:
    condition_a = mean_tpm_per_condition.loc[gene, 'A']
    condition_b = mean_tpm_per_condition.loc[gene, 'B']

    if condition_a > 0 and condition_b > 0:  # Ensure no division by zero
        log2_fc = np.log2(condition_b / condition_a)

        # Compute the t-statistic for the difference
        t_stat = log2_fc / standard_error

        # Degrees of freedom
        df = 1  # Since we are comparing two values

        # Two-tailed p-value from t-distribution
        p_value = 2 * (1 - t.cdf(np.abs(t_stat), df))

        volcano_data['gene_id'].append(gene)
        volcano_data['log2_fold_change'].append(log2_fc)
        volcano_data['p_value'].append(p_value)

        # Debugging print statements
        print(f"Gene: {gene}")
        print(f"Condition A: {condition_a}")
        print(f"Condition B: {condition_b}")
        print(f"Log2 Fold Change: {log2_fc}")
        print(f"P-value: {p_value}")
    else:
        print(f"Skipped gene {gene} due to zero value in condition A or B")

# Convert to DataFrame and add -log10(p-value)
volcano_df = pd.DataFrame(volcano_data)
volcano_df['-log10_p_value'] = -np.log10(volcano_df['p_value'])

# Print the DataFrame to verify
print("First few rows of volcano_df:")
print(volcano_df.head())

# Ensure there are no infinite or NaN values
volcano_df = volcano_df.replace([np.inf, -np.inf], np.nan).dropna()

# Replot the volcano plot
plt.figure(figsize=(10, 8))
plt.scatter(volcano_df['log2_fold_change'], volcano_df['-log10_p_value'], alpha=0.5)
plt.xlabel('Log2 Fold Change')
plt.ylabel('-Log10 P-value')
plt.title('Volcano Plot')
plt.show()


In [ ]:
# Display first few rows of the significant genes DataFrame
print(significant_genes.head())
print(significant_genes.shape)


In [ ]:
# Check the distribution of log2 fold changes
print("Log2 Fold Change Summary:")
print(volcano_df['log2_fold_change'].describe())

# Check the distribution of p-values
print("\nP-value Summary:")
print(volcano_df['p_value'].describe())

# Check the number of significant genes based on different thresholds
volcano_df['is_significant'] = (volcano_df['p_value'] < 0.05) & (volcano_df['log2_fold_change'].abs() >= 1.0)
print("\nNumber of significant genes with log2 fold change >= 1 and p-value < 0.05:")
print(volcano_df['is_significant'].sum())


In [ ]:
# Check the distribution of log2 fold changes
print("Log2 Fold Change Summary:")
print(volcano_df['log2_fold_change'].describe())

# Check the distribution of p-values
print("\nP-value Summary:")
print(volcano_df['p_value'].describe())

# Check the number of significant genes based on different thresholds
volcano_df['is_significant'] = (volcano_df['p_value'] < 0.05) & (volcano_df['log2_fold_change'].abs() >= 1.0)
print("\nNumber of significant genes with log2 fold change >= 1 and p-value < 0.05:")
print(volcano_df['is_significant'].sum())


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plot the distribution of log2 fold changes
plt.figure(figsize=(12, 6))
sns.histplot(volcano_df['log2_fold_change'], bins=50, kde=True)
plt.title('Distribution of Log2 Fold Changes')
plt.xlabel('Log2 Fold Change')
plt.ylabel('Frequency')
plt.show()

# Plot the distribution of p-values
plt.figure(figsize=(12, 6))
sns.histplot(volcano_df['p_value'], bins=50, kde=True)
plt.title('Distribution of P-values')
plt.xlabel('P-value')
plt.ylabel('Frequency')
plt.show()


In [ ]:
# Adjust the thresholds based on the distributions
adjusted_log2_fold_change_threshold = 0.5  # Example adjustment
adjusted_p_value_threshold = 0.1           # Example adjustment

# Identify significant genes based on adjusted thresholds
volcano_df['is_significant'] = (volcano_df['p_value'] < adjusted_p_value_threshold) & (volcano_df['log2_fold_change'].abs() >= adjusted_log2_fold_change_threshold)

# Number of significant genes with adjusted thresholds
print("\nNumber of significant genes with adjusted thresholds:")
print(volcano_df['is_significant'].sum())

# Extract significant genes DataFrame
significant_genes_adjusted = volcano_df[volcano_df['is_significant']]

# Plot the volcano plot with adjusted thresholds
plt.figure(figsize=(10, 8))
sns.scatterplot(data=volcano_df, x='log2_fold_change', y='-log10_p_value', hue='is_significant',
                palette={False: 'blue', True: 'red'}, legend='full', alpha=0.7)
plt.title('Volcano Plot with Adjusted Significant Genes Highlighted')
plt.xlabel('Log2 Fold Change')
plt.ylabel('-Log10 P-Value')
plt.legend(title='Gene Significance', labels=['Not Significant', 'Significant'])
plt.show()



## Data Analysis Summary and Conclusions

### Overview
The dataset comprises gene expression profiles with various measurements such as unstranded, stranded_first, stranded_second, tpm_unstranded, fpkm_unstranded, and fpkm_uq_unstranded. Our analysis focuses on understanding the distribution of these measurements and identifying significant genes based on their log2 fold changes and p-values.

### Key Findings

#### Distribution of TPM (Transcripts Per Million) Unstranded
- The **distribution of `tpm_unstranded`** is highly skewed, with most values concentrated near zero and a few extremely high values (outliers).
- **Boxplot** indicates the presence of numerous outliers, suggesting high variability in gene expression levels among the genes.

#### Log-Transformed Distribution of TPM Unstranded
- **Log-transformation** of `tpm_unstranded` values reveals a more normalized distribution, although there are still a significant number of outliers.
- The **boxplot** of log-transformed `tpm_unstranded` values shows a more compressed range, indicating that log transformation helps in mitigating the impact of outliers.

#### Principal Component Analysis (PCA)
- **PCA plot** reveals clusters of data points with several outliers dispersed away from the main cluster. This indicates variability in gene expression profiles.
- Outliers are highlighted in red, clearly showing the genes with expression levels significantly different from the majority.

#### Volcano Plot
- The **volcano plot** highlights genes with large log2 fold changes and low p-values. Significant genes are expected to be found in the upper corners of the plot.
- In our analysis, no genes were found to be significantly upregulated or downregulated (log2 fold change >= 1 and p-value < 0.05), suggesting that there are no strong candidates for differential expression in the dataset provided.

### Statistical Summary
- **Log2 Fold Change Summary**:
  - The log2 fold change values range from -6.41 to 6.54, with a mean of 0.15 and a standard deviation of 2.02.
- **P-value Summary**:
  - P-values range from 0.097 to 0.978, with a mean of 0.496 and a standard deviation of 0.256.
  - The distribution of p-values is relatively uniform, indicating that there are no strong signals of differential expression.

### Conclusion
- The analysis shows that while there is considerable variability in gene expression levels (as evidenced by the distribution plots and PCA), none of the genes in the current dataset meet the criteria for significant differential expression.
- Future analyses might benefit from increasing the sample size or adjusting the criteria for significance to uncover potential differential expression patterns.
- The insights from the log-transformed distributions and PCA suggest that preprocessing steps such as log transformation are crucial in handling highly skewed data and identifying meaningful patterns.



# Differential Gene Expression Analysis Using BERT for Interpretation

## Step 1: Differential Gene Expression Analysis

### Objective
Identify genes that are differentially expressed between cancerous and non-cancerous tissues.

### Data Preparation




In [ ]:
import pandas as pd

# Load your RNA-seq data
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/processed_rna_seq_data.csv')

# Example of the first few rows of the dataset
print(data.head())
